In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 9.28 ms (started: 2021-08-13 02:19:30 +08:00)


In [3]:
%%bash

# 增加更新
git add *.ipynb

git remote -v

git commit -m '更新 #2 Aug 13, 2021'

git push origin master

origin	git@github.com:ustchope/kerastuner.git (fetch)
origin	git@github.com:ustchope/kerastuner.git (push)
[master 866c520] 更新 #1 Aug 13, 2021
 2 files changed, 153 insertions(+), 51 deletions(-)
 create mode 100644 "\345\210\206\345\270\203\345\274\217\350\266\205\345\217\202\346\225\260\350\260\203\345\217\202.ipynb"


To github.com:ustchope/kerastuner.git
   2a18a28..866c520  master -> master


time: 3.81 s (started: 2021-08-13 02:02:07 +08:00)


In [2]:
#设置使用的gpu
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 1.55 s (started: 2021-08-13 02:19:45 +08:00)


# 介绍

KerasTuner 可以轻松执行分布式超参数搜索。 从在本地运行单线程扩展到在数十或数百个工作线程上并行运行，无需更改您的代码。 分布式 KerasTuner 使用首席工人模型。 负责人运行一个服务，工作人员向该服务报告结果并查询接下来要尝试的超参数。 主管应该在单线程 CPU 实例上运行（或者作为一个单独的进程运行在其中一个工作线程上）。

# 配置分布式模式

为 KerasTuner 配置分布式模式只需要设置三个环境变量：

`KERASTUNER_TUNER_ID`：对于主要进程，这应该设置为“chief”。 其他工人应该被传递一个唯一的 ID（按照惯例，“tuner0”、“tuner1”等）。

`KERASTUNER_ORACLE_IP`：主要服务应该运行的 IP 地址或主机名。 所有工作人员都应该能够解析和访问此地址。

`KERASTUNER_ORACLE_PORT`：主要服务应该运行的端口。 这可以自由选择，但必须是其他工人可以访问的端口。 实例通过 gRPC 协议进行通信。

相同的代码可以在所有工人上运行。 分布式模式的其他注意事项是：
* 所有工作人员都应该可以访问一个集中式文件系统，他们可以将结果写入其中。
* 所有工作人员都应该能够访问调优所需的必要培训和验证数据。
* 为了支持容错，`overwrite`应该在 `Tuner.__init__` 中保持为 `False` （`False` 是默认值）。

首席服务的示例 bash 脚本（页面底部的 run_tuning.py 示例代码）：

In [ ]:
%%bash

export KERASTUNER_TUNER_ID="chief"
export KERASTUNER_ORACLE_IP="127.0.0.1"
export KERASTUNER_ORACLE_PORT="8000"
python run_tuning.py

工人的示例 bash 脚本：

In [ ]:
%%bash

export KERASTUNER_TUNER_ID="tuner0"
export KERASTUNER_ORACLE_IP="127.0.0.1"
export KERASTUNER_ORACLE_PORT="8000"
python run_tuning.py

# tf.distribute 的数据并行性

KerasTuner 还通过 tf.distribute 支持数据并行。 数据并行和分布式调优可以结合起来。 例如，如果您有 10 个 worker，每个 worker 上有 4 个 GPU，您可以使用 tf.distribute.MirroredStrategy 在 4 个 GPU 上运行 10 次并行试验，每次试验训练。 您还可以通过 tf.distribute.experimental.TPUStrategy 在 TPU 上运行每个试验。 目前不支持 tf.distribute.MultiWorkerMirroredStrategy，但对此的支持在路线图上。

# 示例代码

设置上述环境变量后，下面的示例将运行分布式调整并通过 tf.distribute 在每个试验中使用数据并行性。 该示例从 tensorflow_datasets 加载 MNIST 并使用 Hyperband 进行超参数搜索。

In [3]:
import keras_tuner as kt
import tensorflow as tf
import numpy as np


def build_model(hp):
    """Builds a convolutional model."""
    inputs = tf.keras.Input(shape=(28, 28, 1))
    x = inputs
    for i in range(hp.Int("conv_layers", 1, 3, default=3)):
        x = tf.keras.layers.Conv2D(
            filters=hp.Int("filters_" + str(i), 4, 32, step=4, default=8),
            kernel_size=hp.Int("kernel_size_" + str(i), 3, 5),
            activation="relu",
            padding="same",
        )(x)

        if hp.Choice("pooling" + str(i), ["max", "avg"]) == "max":
            x = tf.keras.layers.MaxPooling2D()(x)
        else:
            x = tf.keras.layers.AveragePooling2D()(x)

        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)

    if hp.Choice("global_pooling", ["max", "avg"]) == "max":
        x = tf.keras.layers.GlobalMaxPooling2D()(x)
    else:
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(10, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)

    optimizer = hp.Choice("optimizer", ["adam", "sgd"])
    model.compile(
        optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model


tuner = kt.Hyperband(
    hypermodel=build_model,
    objective="val_accuracy",
    max_epochs=2,
    factor=3,
    hyperband_iterations=1,
    distribution_strategy=tf.distribute.MirroredStrategy(),
    directory="results_dir",
    project_name="mnist",
    overwrite=True,
)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Reshape the images to have the channel dimension.
x_train = (x_train.reshape(x_train.shape + (1,)) / 255.0)[:1000]
y_train = y_train.astype(np.int64)[:1000]
x_test = (x_test.reshape(x_test.shape + (1,)) / 255.0)[:100]
y_test = y_test.astype(np.int64)[:100]

tuner.search(
    x_train,
    y_train,
    steps_per_epoch=600,
    validation_data=(x_test, y_test),
    validation_steps=100,
    callbacks=[tf.keras.callbacks.EarlyStopping("val_accuracy")],
)

Trial 2 Complete [00h 00m 11s]
val_accuracy: 0.18000000715255737

Best val_accuracy So Far: 0.25
Total elapsed time: 00h 00m 27s
INFO:tensorflow:Oracle triggered exit
time: 30.2 s (started: 2021-08-13 02:19:51 +08:00)
